# ANP PDF to Knowledge Graph & Policy Tree

**Objetivo**: Construir um pipeline completo de extração de texto de PDFs normativos da ANP, geração de Knowledge Graph orientado a decisão, projeção em Policy Graph (DAG decisório) e compilação final em árvore JSON compatível com classificador LATS.

**Outputs**:
- `artifacts/anp_text_corpus.jsonl` - Textos limpos por PDF
- `artifacts/anp_kg.graphml` - Knowledge Graph completo
- `artifacts/anp_policy.graphml` - Policy Graph (DAG decisório)
- `artifacts/anp_tree.json` - Árvore de decisão final

**Versão**: 1.0  
**Data**: 2025-12-20

## [0] Setup e Imports

### Dependências necessárias:

```bash
pip install pymupdf pdfplumber pytesseract pillow langchain langchain-experimental langchain-openai networkx pydantic python-dotenv tqdm matplotlib
```

**Nota**: Para OCR, é necessário ter o Tesseract instalado no sistema:
- Ubuntu/Debian: `sudo apt-get install tesseract-ocr tesseract-ocr-por`
- macOS: `brew install tesseract tesseract-lang`
- Windows: Download do instalador em https://github.com/UB-Mannheim/tesseract/wiki

In [45]:
# Imports padrão
import json
import re
import os
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any, Tuple, Optional
from collections import defaultdict, Counter
import hashlib

# Processamento de PDF e OCR
import fitz  # PyMuPDF
try:
    import pytesseract
    from PIL import Image
    OCR_AVAILABLE = True
except ImportError:
    OCR_AVAILABLE = False
    print("⚠️ pytesseract não disponível. OCR será desabilitado.")

# Grafo e análise
import networkx as nx
from tqdm.auto import tqdm

# LangChain e LLM
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document

# Configuração
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports carregados com sucesso")

⚠️ pytesseract não disponível. OCR será desabilitado.
✅ Imports carregados com sucesso


In [46]:
# Configuração de diretórios
PDF_DIR = Path("../padroes_anp")
ARTIFACTS_DIR = Path("../artifacts")

# Criar diretório de artefatos se não existir
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

# Verificar se diretório de PDFs existe
if not PDF_DIR.exists():
    print(f"⚠️ Diretório {PDF_DIR} não encontrado. Criando...")
    PDF_DIR.mkdir(parents=True, exist_ok=True)
else:
    print(f"✅ Diretório de PDFs encontrado: {PDF_DIR}")

# Arquivos de saída
CORPUS_FILE = ARTIFACTS_DIR / "anp_text_corpus.jsonl"
KG_GRAPHML = ARTIFACTS_DIR / "anp_kg.graphml"
KG_JSON = ARTIFACTS_DIR / "anp_kg.json"
POLICY_GRAPHML = ARTIFACTS_DIR / "anp_policy.graphml"
POLICY_JSON = ARTIFACTS_DIR / "anp_policy.json"
TREE_JSON = ARTIFACTS_DIR / "anp_tree.json"

print(f"\n📂 Configuração:")
print(f"   PDFs: {PDF_DIR.absolute()}")
print(f"   Artefatos: {ARTIFACTS_DIR.absolute()}")

✅ Diretório de PDFs encontrado: ../padroes_anp

📂 Configuração:
   PDFs: /home/puppyn/projects/ANP_classifier/notebooks/../padroes_anp
   Artefatos: /home/puppyn/projects/ANP_classifier/notebooks/../artifacts


## [1] Descobrir PDFs

Listar todos os PDFs no diretório e exibir informações básicas.

In [47]:
def descobrir_pdfs(pdf_dir: Path) -> List[Dict[str, Any]]:
    """
    Descobre todos os PDFs no diretório especificado.
    
    Args:
        pdf_dir: Diretório contendo os PDFs
        
    Returns:
        Lista de dicionários com informações sobre cada PDF
    """
    pdfs = []
    
    for pdf_path in sorted(pdf_dir.glob("*.pdf")):
        try:
            doc = fitz.open(pdf_path)
            pdfs.append({
                "path": pdf_path,
                "filename": pdf_path.name,
                "size_mb": pdf_path.stat().st_size / (1024 * 1024),
                "num_pages": len(doc),
                "doc_id": hashlib.md5(pdf_path.name.encode()).hexdigest()[:12]
            })
            doc.close()
        except Exception as e:
            print(f"⚠️ Erro ao abrir {pdf_path.name}: {e}")
    
    return pdfs

# Descobrir PDFs
pdfs_info = descobrir_pdfs(PDF_DIR)

print(f"\n📚 PDFs Encontrados: {len(pdfs_info)}\n")
print(f"{'Filename':<50} {'Páginas':<10} {'Tamanho (MB)':<15} {'Doc ID'}")
print("-" * 90)

for info in pdfs_info:
    print(f"{info['filename']:<50} {info['num_pages']:<10} {info['size_mb']:<15.2f} {info['doc_id']}")

if len(pdfs_info) == 0:
    print("\n⚠️ ATENÇÃO: Nenhum PDF encontrado. Coloque os arquivos PDF em:", PDF_DIR.absolute())


📚 PDFs Encontrados: 2

Filename                                           Páginas    Tamanho (MB)    Doc ID
------------------------------------------------------------------------------------------
manual-comunicacao-incidentes-ANP.pdf              99         3.07            2c8962cfe16d
resolucao-anp-n-882-2022.pdf                       5          1.77            8b0048e9291b


## [2] Extração de Texto

Extrai texto de cada PDF usando PyMuPDF, com fallback para OCR quando necessário.

In [48]:
def extrair_texto_pagina(page, min_chars: int = 30) -> Tuple[str, str]:
    """
    Extrai texto de uma página PDF, com fallback para OCR.
    
    Args:
        page: Página do PyMuPDF
        min_chars: Mínimo de caracteres para considerar extração bem-sucedida
        
    Returns:
        Tupla (texto, método) onde método é "text" ou "ocr"
    """
    # Tentar extração direta de texto
    text = page.get_text("text")
    
    # Se texto for muito curto, tentar OCR
    if len(text.strip()) < min_chars and OCR_AVAILABLE:
        try:
            # Renderizar página como imagem
            pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))  # 2x zoom para melhor OCR
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            # OCR
            text_ocr = pytesseract.image_to_string(img, lang='por')
            
            if len(text_ocr.strip()) > len(text.strip()):
                return text_ocr, "ocr"
        except Exception as e:
            print(f"⚠️ Erro no OCR: {e}")
    
    return text, "text"


def extrair_pdf_completo(pdf_path: Path, doc_id: str) -> Dict[str, Any]:
    """
    Extrai texto completo de um PDF.
    
    Args:
        pdf_path: Caminho do PDF
        doc_id: ID único do documento
        
    Returns:
        Dicionário com metadados e texto extraído
    """
    doc = fitz.open(pdf_path)
    pages_data = []
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        text, method = extrair_texto_pagina(page)
        
        pages_data.append({
            "page": page_num + 1,
            "method": method,
            "text": text
        })
    
    doc.close()
    
    return {
        "doc_id": doc_id,
        "filename": pdf_path.name,
        "pages": pages_data,
        "extracted_at": datetime.now().isoformat()
    }


# Extrair texto de todos os PDFs
print("\n📄 Extraindo texto dos PDFs...\n")

raw_extractions = []
for pdf_info in tqdm(pdfs_info, desc="Processando PDFs"):
    extraction = extrair_pdf_completo(pdf_info["path"], pdf_info["doc_id"])
    raw_extractions.append(extraction)
    
    # Estatísticas
    ocr_pages = sum(1 for p in extraction["pages"] if p["method"] == "ocr")
    if ocr_pages > 0:
        print(f"  {pdf_info['filename']}: {ocr_pages}/{len(extraction['pages'])} páginas via OCR")

print(f"\n✅ Extraídos {len(raw_extractions)} documentos")


📄 Extraindo texto dos PDFs...



Processando PDFs: 100%|██████████| 2/2 [00:00<00:00,  4.36it/s]


✅ Extraídos 2 documentos


## [3] Limpeza e Normalização

Normaliza espaços, remove headers/footers repetitivos, corrige hifenização e problemas de encoding.

In [49]:
def normalize_whitespace(text: str) -> str:
    """
    Normaliza espaços em branco, tabs e quebras de linha.
    
    Args:
        text: Texto a normalizar
        
    Returns:
        Texto normalizado
    """
    # Substituir múltiplos espaços por um único
    text = re.sub(r'[ \t]+', ' ', text)
    # Substituir múltiplas quebras de linha por no máximo duas
    text = re.sub(r'\n{3,}', '\n\n', text)
    # Remover espaços no início e fim de linhas
    text = '\n'.join(line.strip() for line in text.split('\n'))
    return text.strip()


def remove_page_headers_footers(pages: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Remove headers e footers repetitivos detectando linhas que aparecem em muitas páginas.
    
    Args:
        pages: Lista de páginas com texto
        
    Returns:
        Lista de páginas com headers/footers removidos
    """
    if len(pages) < 3:
        return pages
    
    # Coletar primeiras e últimas 3 linhas de cada página
    first_lines = defaultdict(int)
    last_lines = defaultdict(int)
    
    for page in pages:
        lines = page["text"].split('\n')
        if len(lines) > 6:
            for line in lines[:3]:
                line_clean = line.strip()
                if len(line_clean) > 5:  # Ignorar linhas muito curtas
                    first_lines[line_clean] += 1
            for line in lines[-3:]:
                line_clean = line.strip()
                if len(line_clean) > 5:
                    last_lines[line_clean] += 1
    
    # Detectar linhas que aparecem em > 50% das páginas
    threshold = len(pages) * 0.5
    headers = {line for line, count in first_lines.items() if count > threshold}
    footers = {line for line, count in last_lines.items() if count > threshold}
    
    # Remover headers/footers
    cleaned_pages = []
    for page in pages:
        lines = page["text"].split('\n')
        cleaned_lines = [line for line in lines if line.strip() not in headers and line.strip() not in footers]
        
        cleaned_pages.append({
            **page,
            "text": '\n'.join(cleaned_lines)
        })
    
    return cleaned_pages


def dehyphenate(text: str) -> str:
    """
    Junta palavras quebradas por hífen no final de linha.
    
    Args:
        text: Texto a processar
        
    Returns:
        Texto com hifenização corrigida
    """
    # Padrão: hífen no final de linha seguido por quebra e palavra
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
    return text


def fix_encoding_artifacts(text: str) -> str:
    """
    Corrige artefatos comuns de encoding.
    
    Args:
        text: Texto a corrigir
        
    Returns:
        Texto com encoding corrigido
    """
    replacements = {
        'Ã§': 'ç',
        'Ã£': 'ã',
        'Ã©': 'é',
        'Ã¡': 'á',
        'Ã³': 'ó',
        'Ãª': 'ê',
        'Ã´': 'ô',
        'Ã': 'í',
        'Ãº': 'ú',
    }
    
    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)
    
    return text


def clean_document(extraction: Dict[str, Any]) -> Dict[str, Any]:
    """
    Aplica pipeline completo de limpeza em um documento.
    
    Args:
        extraction: Dicionário com extração bruta
        
    Returns:
        Dicionário com texto limpo
    """
    # Remover headers/footers
    pages_cleaned = remove_page_headers_footers(extraction["pages"])
    
    # Concatenar todas as páginas
    full_text = "\n\n".join(page["text"] for page in pages_cleaned)
    
    # Aplicar limpezas
    full_text = fix_encoding_artifacts(full_text)
    full_text = dehyphenate(full_text)
    full_text = normalize_whitespace(full_text)
    
    return {
        "doc_id": extraction["doc_id"],
        "filename": extraction["filename"],
        "text_clean": full_text,
        "num_chars": len(full_text),
        "num_words": len(full_text.split())
    }


# Limpar todos os documentos
print("\n🧹 Limpando e normalizando textos...\n")

clean_docs = []
for extraction in tqdm(raw_extractions, desc="Limpando documentos"):
    clean_doc = clean_document(extraction)
    clean_docs.append(clean_doc)

# Salvar corpus limpo
with open(CORPUS_FILE, 'w', encoding='utf-8') as f:
    for doc in clean_docs:
        f.write(json.dumps(doc, ensure_ascii=False) + '\n')

print(f"\n✅ Corpus salvo em: {CORPUS_FILE}")
print(f"\n📊 Estatísticas do Corpus:")
print(f"   Total de documentos: {len(clean_docs)}")
print(f"   Total de caracteres: {sum(doc['num_chars'] for doc in clean_docs):,}")
print(f"   Total de palavras: {sum(doc['num_words'] for doc in clean_docs):,}")
print(f"   Média palavras/doc: {sum(doc['num_words'] for doc in clean_docs) / len(clean_docs):.0f}")


🧹 Limpando e normalizando textos...



Limpando documentos: 100%|██████████| 2/2 [00:00<00:00, 34.94it/s]


✅ Corpus salvo em: ../artifacts/anp_text_corpus.jsonl

📊 Estatísticas do Corpus:
   Total de documentos: 2
   Total de caracteres: 169,291
   Total de palavras: 24,923
   Média palavras/doc: 12462


## [4] Chunking

Divide documentos em chunks baseados em seções detectadas ou por tamanho fixo.

In [50]:
def detectar_secoes(text: str) -> List[Tuple[int, str, str]]:
    """
    Detecta seções no texto usando heurísticas.
    
    Args:
        text: Texto a processar
        
    Returns:
        Lista de tuplas (posição, tipo_secao, título)
    """
    secoes = []
    
    # Padrões de títulos
    patterns = [
        (r'^(CAPÍTULO|SEÇÃO|ANEXO|ARTIGO)\s+[IVX\d]+', 'capitulo'),
        (r'^Art\.\s*\d+', 'artigo'),
        (r'^\d+\.\s+[A-ZÀÁÃÂÉÊÍÓÔÕÚ][A-ZÀÁÃÂÉÊÍÓÔÕÚ\s]{5,}$', 'titulo_caps'),
        (r'^[A-ZÀÁÃÂÉÊÍÓÔÕÚ][A-ZÀÁÃÂÉÊÍÓÔÕÚ\s]{10,}$', 'secao_caps'),
    ]
    
    lines = text.split('\n')
    pos = 0
    
    for line in lines:
        line_stripped = line.strip()
        for pattern, tipo in patterns:
            if re.match(pattern, line_stripped, re.MULTILINE):
                secoes.append((pos, tipo, line_stripped))
                break
        pos += len(line) + 1  # +1 para o \n
    
    return secoes


def chunk_por_secoes(doc: Dict[str, Any], max_chunk_chars: int = 6000) -> List[Dict[str, Any]]:
    """
    Divide documento em chunks baseados em seções detectadas.
    
    Args:
        doc: Documento limpo
        max_chunk_chars: Tamanho máximo de chunk (em caracteres)
        
    Returns:
        Lista de chunks
    """
    text = doc["text_clean"]
    secoes = detectar_secoes(text)
    
    chunks = []
    
    if len(secoes) == 0:
        # Fallback: chunk por tamanho fixo
        for i in range(0, len(text), max_chunk_chars):
            chunk_text = text[i:i + max_chunk_chars]
            chunks.append({
                "chunk_id": f"{doc['doc_id']}_chunk_{len(chunks)}",
                "doc_id": doc["doc_id"],
                "section_hint": "chunk_fixo",
                "text": chunk_text
            })
    else:
        # Chunk por seções
        for i, (pos, tipo, titulo) in enumerate(secoes):
            # Encontrar fim da seção (início da próxima ou fim do texto)
            if i < len(secoes) - 1:
                end_pos = secoes[i + 1][0]
            else:
                end_pos = len(text)
            
            chunk_text = text[pos:end_pos].strip()
            
            # Se chunk muito grande, dividir
            if len(chunk_text) > max_chunk_chars:
                for j in range(0, len(chunk_text), max_chunk_chars):
                    sub_chunk = chunk_text[j:j + max_chunk_chars]
                    chunks.append({
                        "chunk_id": f"{doc['doc_id']}_sec_{i}_part_{j // max_chunk_chars}",
                        "doc_id": doc["doc_id"],
                        "section_hint": f"{tipo}:{titulo[:50]}",
                        "text": sub_chunk
                    })
            else:
                chunks.append({
                    "chunk_id": f"{doc['doc_id']}_sec_{i}",
                    "doc_id": doc["doc_id"],
                    "section_hint": f"{tipo}:{titulo[:50]}",
                    "text": chunk_text
                })
    
    return chunks


# Gerar chunks de todos os documentos
print("\n✂️ Gerando chunks...\n")

all_chunks = []
for doc in tqdm(clean_docs, desc="Chunking documentos"):
    doc_chunks = chunk_por_secoes(doc)
    all_chunks.extend(doc_chunks)
    print(f"  {doc['filename']}: {len(doc_chunks)} chunks")

print(f"\n✅ Total de chunks: {len(all_chunks)}")
print(f"   Tamanho médio: {sum(len(c['text']) for c in all_chunks) / len(all_chunks):.0f} caracteres")


✂️ Gerando chunks...



Chunking documentos: 100%|██████████| 2/2 [00:00<00:00, 106.12it/s]

  manual-comunicacao-incidentes-ANP.pdf: 43 chunks
  resolucao-anp-n-882-2022.pdf: 0 chunks

✅ Total de chunks: 43
   Tamanho médio: 3934 caracteres


## [5] LLM Config (Azure OpenAI)

Configurar conexão com Azure OpenAI para geração do Knowledge Graph.

In [52]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# --------------------------------------------------
# Carregar variáveis de ambiente
# --------------------------------------------------
load_dotenv()

# Verificar configuração
required_env_vars = ["OPENAI_API_KEY"]

missing_vars = [var for var in required_env_vars if not os.getenv(var)]

if missing_vars:
    print("❌ Variáveis de ambiente faltando:")
    for var in missing_vars:
        print(f"   - {var}")
    print("\n⚠️ Configure o arquivo .env antes de continuar.")
else:
    print("✅ Variáveis de ambiente carregadas")

# --------------------------------------------------
# Instanciar LLM (OpenAI direto)
# --------------------------------------------------
llm = ChatOpenAI(
    model="gpt-4o-mini",   # pode trocar para "gpt-4o" se quiser
    temperature=0,
    max_tokens=4000
)

print("\n🤖 LLM configurado:")
print("   Provider: OpenAI")
print("   Model: gpt-4o-mini")
print("   Temperature: 0 (determinístico)")


✅ Variáveis de ambiente carregadas

🤖 LLM configurado:
   Provider: OpenAI
   Model: gpt-4o-mini
   Temperature: 0 (determinístico)


## [6] Knowledge Graph com LLMGraphTransformer

Gera Knowledge Graph orientado a decisão usando LLM para extrair entidades e relações normativas.

In [53]:
# Schema decisional para o KG
ALLOWED_NODES = [
    "IncidentType",      # Tipo de incidente (lesão, meio ambiente, etc)
    "Criterion",         # Critério decisório (pergunta)
    "Threshold",         # Limiar numérico (volume, dias, etc)
    "Classification",    # Classificação final (Classe 1, 2, etc)
    "Obligation",        # Obrigação normativa
    "Exception",         # Exceção à regra
    "Actor",            # Ator envolvido (ANP, operador, etc)
    "Evidence"          # Evidência necessária
]

ALLOWED_RELATIONSHIPS = [
    "DEPENDS_ON",        # Critério depende de outro
    "CLASSIFIED_AS",     # Leva à classificação
    "IMPLIES",           # Implica consequência
    "REQUIRES",          # Requer evidência/ação
    "HAS_THRESHOLD",     # Possui limiar
    "HAS_EXCEPTION",     # Possui exceção
    "APPLIES_TO",        # Aplica-se a
    "EVIDENCED_BY"       # Evidenciado por
]

# Prompt guia para extração decisional
DECISIONAL_GUIDE = """Extraia apenas conceitos necessários para CLASSIFICAR INCIDENTES segundo a norma ANP.

FOQUE EM:
- Critérios perguntáveis (sim/não, qual tipo, qual faixa)
- Thresholds numéricos (volume > X, dias >= Y)
- Exceções e condições especiais
- Mapeamento direto para classificações (Classe 1, 2, 3, etc)

IGNORE:
- Narrativa histórica
- Exemplos irrelevantes para decisão
- Contexto administrativo geral
- Definições que não afetam classificação

IMPORTANTE: Mantenha foco em construir um grafo DECISIONAL, não enciclopédico.
"""

# Configurar transformer
graph_transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=ALLOWED_NODES,
    allowed_relationships=ALLOWED_RELATIONSHIPS,
    node_properties=True,
    relationship_properties=True
)

print("✅ LLMGraphTransformer configurado")
print(f"\n📋 Schema Decisional:")
print(f"   Tipos de nós: {len(ALLOWED_NODES)}")
print(f"   Tipos de relações: {len(ALLOWED_RELATIONSHIPS)}")

✅ LLMGraphTransformer configurado

📋 Schema Decisional:
   Tipos de nós: 8
   Tipos de relações: 8


In [54]:
def processar_chunks_para_kg(chunks: List[Dict[str, Any]], 
                             transformer: LLMGraphTransformer,
                             max_chunks: Optional[int] = None) -> nx.DiGraph:
    """
    Processa chunks e gera Knowledge Graph unificado.
    
    Args:
        chunks: Lista de chunks de texto
        transformer: LLMGraphTransformer configurado
        max_chunks: Limite de chunks a processar (para testes)
        
    Returns:
        NetworkX DiGraph com KG unificado
    """
    kg = nx.DiGraph()
    
    # Limitar chunks se especificado
    chunks_to_process = chunks[:max_chunks] if max_chunks else chunks
    
    print(f"\n🧠 Processando {len(chunks_to_process)} chunks com LLM...\n")
    
    for chunk in tqdm(chunks_to_process, desc="Gerando KG"):
        try:
            # Preparar documento com guia decisional
            doc_text = f"{DECISIONAL_GUIDE}\n\n{chunk['text']}"
            doc = Document(page_content=doc_text, metadata={"chunk_id": chunk["chunk_id"]})
            
            # Extrair grafo do chunk
            graph_docs = transformer.convert_to_graph_documents([doc])
            
            # Adicionar ao KG unificado
            for graph_doc in graph_docs:
                # Adicionar nós
                for node in graph_doc.nodes:
                    node_id = f"{node.type}:{node.id}"
                    if node_id not in kg:
                        kg.add_node(node_id, 
                                   type=node.type, 
                                   id=node.id,
                                   properties=node.properties if hasattr(node, 'properties') else {})
                
                # Adicionar arestas
                for rel in graph_doc.relationships:
                    source_id = f"{rel.source.type}:{rel.source.id}"
                    target_id = f"{rel.target.type}:{rel.target.id}"
                    
                    if not kg.has_edge(source_id, target_id):
                        kg.add_edge(source_id, target_id,
                                   type=rel.type,
                                   properties=rel.properties if hasattr(rel, 'properties') else {})
        
        except Exception as e:
            print(f"⚠️ Erro ao processar chunk {chunk['chunk_id']}: {e}")
            continue
    
    return kg


# Gerar KG (NOTA: Processar todos os chunks pode levar muito tempo e custar muito)
# Para teste inicial, processar apenas os primeiros N chunks
TEST_MODE = True  # Altere para False para processar tudo
MAX_CHUNKS_TEST = 10

if TEST_MODE:
    print(f"\n⚠️ MODO DE TESTE: Processando apenas {MAX_CHUNKS_TEST} chunks")
    print("   Altere TEST_MODE = False para processar corpus completo\n")
    kg = processar_chunks_para_kg(all_chunks, graph_transformer, max_chunks=MAX_CHUNKS_TEST)
else:
    kg = processar_chunks_para_kg(all_chunks, graph_transformer)

print(f"\n✅ Knowledge Graph gerado:")
print(f"   Nós: {kg.number_of_nodes()}")
print(f"   Arestas: {kg.number_of_edges()}")

# Estatísticas por tipo
node_types = Counter(kg.nodes[n]['type'] for n in kg.nodes())
print(f"\n📊 Distribuição de Nós:")
for node_type, count in node_types.most_common():
    print(f"   {node_type}: {count}")


⚠️ MODO DE TESTE: Processando apenas 10 chunks
   Altere TEST_MODE = False para processar corpus completo


🧠 Processando 10 chunks com LLM...



Gerando KG: 100%|██████████| 10/10 [01:35<00:00,  9.60s/it]


✅ Knowledge Graph gerado:
   Nós: 39
   Arestas: 45

📊 Distribuição de Nós:
   Criterion: 14
   Classification: 11
   Threshold: 5
   Incidenttype: 5
   Exception: 3
   Actor: 1


In [56]:
# Exportar KG
print("\n💾 Salvando Knowledge Graph...")

# JSON (node-link format)
kg_json = nx.node_link_data(kg)
with open(KG_JSON, 'w', encoding='utf-8') as f:
    json.dump(kg_json, f, ensure_ascii=False, indent=2)
print(f"   ✅ JSON: {KG_JSON}")


💾 Salvando Knowledge Graph...
   ✅ JSON: ../artifacts/anp_kg.json


## [7] Policy Graph (Projeção do KG)

Projeta o KG em um Policy Graph decisório, focando em critérios, thresholds e classificações.

In [57]:
def projetar_policy_graph(kg: nx.DiGraph) -> nx.DiGraph:
    """
    Projeta KG em Policy Graph decisório.
    
    Args:
        kg: Knowledge Graph completo
        
    Returns:
        Policy Graph (DAG)
    """
    policy = nx.DiGraph()
    
    # Tipos de nós relevantes para decisão
    decision_node_types = {"IncidentType", "Criterion", "Threshold", "Exception", "Classification"}
    
    # Tipos de relações relevantes
    decision_edge_types = {"DEPENDS_ON", "HAS_THRESHOLD", "HAS_EXCEPTION", "IMPLIES", "REQUIRES", "CLASSIFIED_AS"}
    
    # Adicionar nós decisórios
    for node_id, data in kg.nodes(data=True):
        if data.get('type') in decision_node_types:
            policy.add_node(node_id, **data)
    
    # Adicionar arestas decisórias
    for source, target, data in kg.edges(data=True):
        if data.get('type') in decision_edge_types:
            if source in policy and target in policy:
                policy.add_edge(source, target, **data)
    
    # Transformar Thresholds em Criterions quando necessário
    threshold_nodes = [n for n in policy.nodes() if policy.nodes[n]['type'] == 'Threshold']
    
    for threshold_id in threshold_nodes:
        # Criar criterion associado
        threshold_data = policy.nodes[threshold_id]
        criterion_id = threshold_id.replace('Threshold:', 'Criterion:')
        
        if criterion_id not in policy:
            policy.add_node(criterion_id,
                          type='Criterion',
                          id=threshold_data['id'],
                          properties={'derived_from_threshold': threshold_id})
            
            # Conectar criterion ao threshold
            policy.add_edge(criterion_id, threshold_id, type='HAS_THRESHOLD')
    
    return policy


def validar_e_corrigir_dag(graph: nx.DiGraph) -> nx.DiGraph:
    """
    Valida se é DAG e remove ciclos se necessário.
    
    Args:
        graph: Grafo a validar
        
    Returns:
        DAG válido
    """
    if nx.is_directed_acyclic_graph(graph):
        print("✅ Grafo é DAG válido")
        return graph
    
    print("⚠️ Grafo contém ciclos. Removendo...")
    
    # Encontrar ciclos
    try:
        cycles = list(nx.simple_cycles(graph))
        print(f"   Encontrados {len(cycles)} ciclos")
        
        # Remover arestas IMPLIES de ciclos (menor impacto)
        for cycle in cycles:
            # Encontrar aresta IMPLIES no ciclo
            for i in range(len(cycle)):
                source = cycle[i]
                target = cycle[(i + 1) % len(cycle)]
                
                if graph.has_edge(source, target):
                    edge_type = graph[source][target].get('type')
                    if edge_type == 'IMPLIES':
                        graph.remove_edge(source, target)
                        print(f"   Removida aresta {source} -> {target}")
                        break
    except Exception as e:
        print(f"⚠️ Erro ao processar ciclos: {e}")
    
    # Verificar novamente
    if nx.is_directed_acyclic_graph(graph):
        print("✅ DAG corrigido com sucesso")
    else:
        print("⚠️ Grafo ainda contém ciclos após correção")
    
    return graph


# Gerar Policy Graph
print("\n🎯 Gerando Policy Graph...\n")

policy_graph = projetar_policy_graph(kg)

print(f"\n📊 Policy Graph:")
print(f"   Nós: {policy_graph.number_of_nodes()}")
print(f"   Arestas: {policy_graph.number_of_edges()}")

# Estatísticas por tipo
policy_node_types = Counter(policy_graph.nodes[n]['type'] for n in policy_graph.nodes())
print(f"\n📊 Distribuição de Nós (Policy):")
for node_type, count in policy_node_types.most_common():
    print(f"   {node_type}: {count}")

# Validar DAG
print("\n🔍 Validando DAG...")
policy_graph = validar_e_corrigir_dag(policy_graph)


🎯 Gerando Policy Graph...


📊 Policy Graph:
   Nós: 38
   Arestas: 14

📊 Distribuição de Nós (Policy):
   Criterion: 19
   Classification: 11
   Threshold: 5
   Exception: 3

🔍 Validando DAG...
✅ Grafo é DAG válido


In [58]:
# Exportar Policy Graph
print("\n💾 Salvando Policy Graph...")

# JSON
policy_json = nx.node_link_data(policy_graph)
with open(POLICY_JSON, 'w', encoding='utf-8') as f:
    json.dump(policy_json, f, ensure_ascii=False, indent=2)
print(f"   ✅ JSON: {POLICY_JSON}")


💾 Salvando Policy Graph...
   ✅ JSON: ../artifacts/anp_policy.json


## [8] Compilação Policy → Árvore com Subpolicies

Compila Policy Graph em árvore JSON com ramificação por subpolicies detectadas automaticamente.

**Estratégia**:
1. Criar nó raiz roteador ("Qual o tipo de ocorrência?")
2. Para cada subpolicy: extrair subgrafo e compilar subárvore
3. Anexar subárvores como ramos da raiz
4. Resultado: árvore com branching semântico, menor entropia

In [61]:
from typing import Dict, Any, List, Tuple
import networkx as nx
import json

# 🔹 Utilitário: ordenar nós decisórios dentro da subpolicy

def ordenar_nos_decisao(subgraph: nx.DiGraph) -> List[str]:
    """
    Define ordem decisória dentro de uma subpolicy.
    Prioriza precedência estrutural (topological sort).
    """
    decision_nodes = [
        n for n, d in subgraph.nodes(data=True)
        if d.get("type") in {"Criterion", "Threshold"}
    ]

    decision_subgraph = subgraph.subgraph(decision_nodes)

    try:
        return list(nx.topological_sort(decision_subgraph))
    except nx.NetworkXUnfeasible:
        # Fallback: nós mais conectados primeiro
        return sorted(decision_nodes, key=lambda n: subgraph.degree(n), reverse=True)

#🔹 Construção recursiva de subnodos (mantida, mas agora com fluxo)

def construir_subnodos(
    policy: nx.DiGraph,
    node_id: str,
    visited: set = None,
    depth: int = 0,
    max_depth: int = 10
) -> List[Dict[str, Any]]:
    """
    Constrói subnodos recursivamente com encerramento local.
    """
    if visited is None:
        visited = set()

    if depth > max_depth or node_id in visited:
        return []

    visited.add(node_id)
    subnodos = []

    # 1️⃣ Encerramentos locais
    for succ in policy.successors(node_id):
        if policy.nodes[succ].get("type") == "Classification":
            subnodos.append({
                "id": succ.replace(':', '_').replace(' ', '_').lower(),
                "tipo": "terminal",
                "classe": policy.nodes[succ].get("id", "Classe não especificada")
            })

    # 2️⃣ Continuação do fluxo
    for succ in policy.successors(node_id):
        succ_type = policy.nodes[succ].get("type")

        if succ_type in {"Criterion", "Threshold"}:
            children = construir_subnodos(
                policy,
                succ,
                visited.copy(),
                depth + 1,
                max_depth
            )

            if not children:
                children = [{
                    "id": f"{succ}_default".replace(':', '_').replace(' ', '_').lower(),
                    "tipo": "terminal",
                    "classe": "Requer análise adicional"
                }]

            subnodos.append({
                "id": succ.replace(':', '_').replace(' ', '_').lower(),
                "pergunta": policy.nodes[succ].get("id", "Critério"),
                "tipo": "decisao",
                "subnodos": children
            })

    return subnodos
# 🔹 Compilação de uma subpolicy (CORRIGIDA)

def compilar_subarvore(policy_subgraph: nx.DiGraph, subpolicy_id: str) -> Dict[str, Any]:
    """
    Compila um subgrafo (subpolicy) em subárvore com ordem decisória explícita.
    """

    ordered_nodes = ordenar_nos_decisao(policy_subgraph)

    if not ordered_nodes:
        return {
            "id": subpolicy_id,
            "tipo": "terminal",
            "classe": "Subpolicy sem critérios executáveis"
        }

    root_node_id = ordered_nodes[0]
    root_data = policy_subgraph.nodes[root_node_id]

    subpolicy_root = {
        "id": subpolicy_id,
        "pergunta": root_data.get("id", f"Domínio normativo {subpolicy_id}"),
        "tipo": "decisao",
        "subnodos": construir_subnodos(
            policy_subgraph,
            root_node_id,
            visited=set(),
            depth=0
        )
    }

    if not subpolicy_root["subnodos"]:
        subpolicy_root["subnodos"].append({
            "id": f"{subpolicy_id}_terminal",
            "tipo": "terminal",
            "classe": "Requer análise técnica específica"
        })

    return subpolicy_root
#🔹 Compilador final com subpolicies (plug-and-play)

def compilar_arvore_com_subpolicies(
    policy: nx.DiGraph,
    communities: List[set]
) -> Dict[str, Any]:
    """
    Compila Policy Graph em árvore com ramificação automática por subpolicies.
    """

    raiz = {
        "id": "raiz",
        "pergunta": "Qual o tipo de ocorrência?",
        "tipo": "decisao",
        "subnodos": []
    }

    print(f"\n🌳 Compilando árvore com {len(communities)} subpolicies...\n")

    for i, community in enumerate(communities):
        subpolicy_id = f"subpolicy_{i}"

        nodes_in_subpolicy = set(community)

        # incluir classificações associadas
        for n in community:
            for succ in policy.successors(n):
                if policy.nodes[succ].get("type") == "Classification":
                    nodes_in_subpolicy.add(succ)

        subgraph = policy.subgraph(nodes_in_subpolicy).copy()

        print(f"   Subpolicy {i}:")
        print(f"      Nós: {subgraph.number_of_nodes()}")
        print(f"      Arestas: {subgraph.number_of_edges()}")

        subarvore = compilar_subarvore(subgraph, subpolicy_id)
        raiz["subnodos"].append(subarvore)

    if not raiz["subnodos"]:
        raiz["subnodos"].append({
            "id": "incidente_generico",
            "tipo": "terminal",
            "classe": "Classificação não determinada"
        })

    return raiz



## [7.5] Detecção Automática de Subpolicies

Utiliza expansão controlada baseada em **âncoras normativas** (IncidentType e Classification) para detectar subpolicies no Policy Graph.

**Abordagem**:
- **Âncoras Normativas**: Nós do tipo IncidentType e Classification servem como pontos de partida
- **Expansão Controlada**: Navegação bidirecional (predecessores + sucessores) com profundidade limitada
- **Evita Duplicação**: Subpolicies com pouca novidade (< 5 nós novos) são descartadas

**Objetivo**: Ramificar árvore por domínios normativos detectados automaticamente, reduzindo entropia.

In [ ]:
#🔹 1. Preparar grafo para detecção de comunidades
print("\n🧠 Detectando subpolicies automaticamente...\n")

def detectar_ancoras_normativas(policy: nx.DiGraph):
    anchors = []

    for n, data in policy.nodes(data=True):
        if data.get("type") in ["IncidentType", "Classification"]:
            anchors.append(n)

    return anchors

def expandir_subpolicy(policy: nx.DiGraph, anchor: str, max_depth=5):
    visited = set()
    queue = [(anchor, 0)]
    subpolicy_nodes = set([anchor])

    while queue:
        node, depth = queue.pop(0)
        if depth >= max_depth:
            continue

        for succ in policy.successors(node):
            if succ not in visited:
                visited.add(succ)
                subpolicy_nodes.add(succ)
                queue.append((succ, depth + 1))

        for pred in policy.predecessors(node):
            if pred not in visited:
                visited.add(pred)
                subpolicy_nodes.add(pred)
                queue.append((pred, depth + 1))

    return subpolicy_nodes

anchors = detectar_ancoras_normativas(policy_graph)

subpolicies = []
used_nodes = set()

for i, anchor in enumerate(anchors):
    nodes = expandir_subpolicy(policy_graph, anchor)

    # Evitar subpolicies duplicadas
    if len(nodes - used_nodes) < 5:
        continue

    subpolicies.append(nodes)
    used_nodes |= nodes


🧠 Detectando subpolicies automaticamente...



In [ ]:
#🔹 Execução + estatísticas

print("\n🌳 Compilando árvore de decisão com subpolicies...\n")

arvore_decisao = compilar_arvore_com_subpolicies(policy_graph, subpolicies)

with open(TREE_JSON, 'w', encoding='utf-8') as f:
    json.dump(arvore_decisao, f, ensure_ascii=False, indent=2)

print(f"\n✅ Árvore de decisão salva em: {TREE_JSON}")

def contar_nos_recursivo(node: Dict[str, Any]) -> Tuple[int, int]:
    if node.get("tipo") == "terminal":
        return 0, 1

    decisao, terminal = 1, 0
    for sub in node.get("subnodos", []):
        d, t = contar_nos_recursivo(sub)
        decisao += d
        terminal += t
    return decisao, terminal


num_decisao, num_terminal = contar_nos_recursivo(arvore_decisao)

print(f"\n📊 Estatísticas da Árvore (COM SUBPOLICIES):")
print(f"   Nós de decisão: {num_decisao}")
print(f"   Nós terminais: {num_terminal}")
print(f"   Total: {num_decisao + num_terminal}")
print(f"   Subpolicies (ramos principais): {len(subpolicies)}")

## [8] Compilação Policy → Árvore

Compila Policy Graph em árvore JSON compatível com classificador modular.

In [64]:
def compilar_arvore_decisao(policy: nx.DiGraph) -> Dict[str, Any]:
    """
    Compila Policy Graph em árvore de decisão JSON.
    
    Args:
        policy: Policy Graph (DAG)
        
    Returns:
        Árvore de decisão em formato JSON
    """
    # Encontrar roots (IncidentTypes sem predecessores)
    roots = [n for n in policy.nodes() 
            if policy.nodes[n]['type'] == 'IncidentType' 
            and policy.in_degree(n) == 0]
    
    if not roots:
        # Fallback: pegar todos IncidentTypes
        roots = [n for n in policy.nodes() if policy.nodes[n]['type'] == 'IncidentType']
    
    # Criar nó raiz "Qual o tipo de ocorrência?"
    raiz = {
        "id": "raiz",
        "pergunta": "Qual o tipo de ocorrência?",
        "tipo": "decisao",
        "subnodos": []
    }
    
    # Para cada IncidentType, criar subárvore
    for root_id in roots:
        root_data = policy.nodes[root_id]
        
        # Criar nó para este tipo de incidente
        incident_node = {
            "id": root_id.replace(':', '_').replace(' ', '_').lower(),
            "pergunta": root_data.get('id', 'Tipo de incidente'),
            "tipo": "decisao",
            "subnodos": []
        }
        
        # Construir subárvore a partir deste nó
        subnodos = construir_subnodos(policy, root_id)
        incident_node["subnodos"] = subnodos
        
        raiz["subnodos"].append(incident_node)
    
    # Se não houver roots, criar estrutura mínima
    if not raiz["subnodos"]:
        raiz["subnodos"].append({
            "id": "incidente_generico",
            "pergunta": "Incidente genérico",
            "tipo": "decisao",
            "subnodos": [
                {
                    "id": "classe_desconhecida",
                    "tipo": "terminal",
                    "classe": "Classificação não determinada"
                }
            ]
        })
    
    return raiz


def construir_subnodos(policy: nx.DiGraph, node_id: str, visited: set = None, depth: int = 0) -> List[Dict[str, Any]]:
    """
    Constrói subnodos recursivamente a partir de um nó.
    
    Args:
        policy: Policy Graph
        node_id: ID do nó atual
        visited: Conjunto de nós já visitados (evitar ciclos)
        depth: Profundidade atual (limitar recursão)
        
    Returns:
        Lista de subnodos
    """
    if visited is None:
        visited = set()
    
    # Limitar profundidade para evitar recursão infinita
    if depth > 10 or node_id in visited:
        return []
    
    visited.add(node_id)
    subnodos = []
    
    # Obter sucessores (ordenados por tipo de relação)
    successors = list(policy.successors(node_id))
    
    for succ_id in successors:
        succ_data = policy.nodes[succ_id]
        edge_data = policy[node_id][succ_id]
        
        # Se sucessor é Classification, criar nó terminal
        if succ_data['type'] == 'Classification':
            subnodos.append({
                "id": succ_id.replace(':', '_').replace(' ', '_').lower(),
                "tipo": "terminal",
                "classe": succ_data.get('id', 'Classe não especificada')
            })
        
        # Se sucessor é Criterion ou Threshold, criar nó de decisão
        elif succ_data['type'] in ['Criterion', 'Threshold']:
            # Construir pergunta
            pergunta = succ_data.get('id', 'Critério')
            
            # Recursivamente construir subnodos
            sub_subnodos = construir_subnodos(policy, succ_id, visited.copy(), depth + 1)
            
            # Se não houver subnodos, criar terminal padrão
            if not sub_subnodos:
                sub_subnodos = [{
                    "id": f"{succ_id}_default".replace(':', '_').replace(' ', '_').lower(),
                    "tipo": "terminal",
                    "classe": "Requer análise adicional"
                }]
            
            subnodos.append({
                "id": succ_id.replace(':', '_').replace(' ', '_').lower(),
                "pergunta": pergunta,
                "tipo": "decisao",
                "subnodos": sub_subnodos
            })
    
    return subnodos


# Compilar árvore
print("\n🌳 Compilando Policy Graph em árvore de decisão...\n")

arvore_decisao = compilar_arvore_decisao(policy_graph)

# Salvar árvore
with open(TREE_JSON, 'w', encoding='utf-8') as f:
    json.dump(arvore_decisao, f, ensure_ascii=False, indent=2)

print(f"✅ Árvore de decisão salva em: {TREE_JSON}")

# Estatísticas da árvore
def contar_nos_recursivo(node: Dict[str, Any]) -> Tuple[int, int]:
    """Conta nós de decisão e terminais recursivamente."""
    if node.get('tipo') == 'terminal':
        return 0, 1
    
    decisao = 1
    terminal = 0
    
    for subnode in node.get('subnodos', []):
        d, t = contar_nos_recursivo(subnode)
        decisao += d
        terminal += t
    
    return decisao, terminal

num_decisao, num_terminal = contar_nos_recursivo(arvore_decisao)

print(f"\n📊 Estatísticas da Árvore:")
print(f"   Nós de decisão: {num_decisao}")
print(f"   Nós terminais: {num_terminal}")
print(f"   Total: {num_decisao + num_terminal}")


🌳 Compilando Policy Graph em árvore de decisão...

✅ Árvore de decisão salva em: ../artifacts/anp_tree.json

📊 Estatísticas da Árvore:
   Nós de decisão: 2
   Nós terminais: 1
   Total: 3


## [9] Relatório de Qualidade

Gera estatísticas e visualizações sobre os artefatos criados.

In [ ]:
print("\n" + "="*80)
print("📊 RELATÓRIO DE QUALIDADE")
print("="*80)

# 1. Corpus
print("\n1️⃣ CORPUS")
print(f"   Documentos processados: {len(clean_docs)}")
print(f"   Chunks gerados: {len(all_chunks)}")
print(f"   Média chunks/doc: {len(all_chunks) / len(clean_docs):.1f}")

# 2. Knowledge Graph
print("\n2️⃣ KNOWLEDGE GRAPH")
print(f"   Total de nós: {kg.number_of_nodes()}")
print(f"   Total de arestas: {kg.number_of_edges()}")
print(f"   Densidade: {nx.density(kg):.4f}")

print("\n   Distribuição de nós por tipo:")
for node_type, count in node_types.most_common():
    print(f"      {node_type}: {count}")

edge_types = Counter(kg[u][v]['type'] for u, v in kg.edges())
print("\n   Distribuição de arestas por tipo:")
for edge_type, count in edge_types.most_common():
    print(f"      {edge_type}: {count}")

# 3. Policy Graph
print("\n3️⃣ POLICY GRAPH")
print(f"   Total de nós: {policy_graph.number_of_nodes()}")
print(f"   Total de arestas: {policy_graph.number_of_edges()}")
print(f"   É DAG: {'✅' if nx.is_directed_acyclic_graph(policy_graph) else '❌'}")

if policy_graph.number_of_nodes() > 0:
    print("\n   Distribuição de nós por tipo:")
    for node_type, count in policy_node_types.most_common():
        print(f"      {node_type}: {count}")

# 3.5. Detecção de Subpolicies (Normativa-Semântica)
print("\n3️⃣.5️⃣ DETECÇÃO DE SUBPOLICIES (ÂNCORAS NORMATIVAS)")
print(f"   Subpolicies detectadas: {len(subpolicies)}")
print(f"   Âncoras encontradas: {len(anchors)}")

print("\n   Distribuição de tamanho das subpolicies:")
for i, sp in enumerate(subpolicies):
    print(f"      subpolicy_{i}: {len(sp)} nós")

# 4. Árvore de Decisão
print("\n4️⃣ ÁRVORE DE DECISÃO (COM SUBPOLICIES)")
print(f"   Nós de decisão: {num_decisao}")
print(f"   Nós terminais (classes): {num_terminal}")
print(f"   Subpolicies (ramos principais): {len(subpolicies)}")

# Profundidade média
def calcular_profundidade_media(node: Dict[str, Any], depth: int = 0) -> List[int]:
    """Calcula profundidades de todos os nós terminais."""
    if node.get('tipo') == 'terminal':
        return [depth]
    
    depths = []
    for subnode in node.get('subnodos', []):
        depths.extend(calcular_profundidade_media(subnode, depth + 1))
    
    return depths

depths = calcular_profundidade_media(arvore_decisao)
if depths:
    print(f"   Profundidade média: {sum(depths) / len(depths):.1f}")
    print(f"   Profundidade mínima: {min(depths)}")
    print(f"   Profundidade máxima: {max(depths)}")

# Fator de ramificação (branching factor) médio
def calcular_branching_factor(node: Dict[str, Any]) -> List[int]:
    """Calcula branching factor de todos os nós de decisão."""
    if node.get('tipo') == 'terminal':
        return []
    
    factors = [len(node.get('subnodos', []))]
    
    for subnode in node.get('subnodos', []):
        factors.extend(calcular_branching_factor(subnode))
    
    return factors

branching_factors = calcular_branching_factor(arvore_decisao)
if branching_factors:
    print(f"   Branching factor médio: {sum(branching_factors) / len(branching_factors):.1f}")
    print(f"   Branching factor máximo: {max(branching_factors)}")

# 5. Critérios mais centrais (por degree no Policy Graph)
if policy_graph.number_of_nodes() > 0:
    print("\n5️⃣ TOP 20 CRITÉRIOS MAIS CENTRAIS (por degree)")
    
    criterion_nodes_list = [n for n in policy_graph.nodes() if policy_graph.nodes[n]['type'] == 'Criterion']
    
    if criterion_nodes_list:
        degrees = [(n, policy_graph.degree(n)) for n in criterion_nodes_list]
        degrees.sort(key=lambda x: x[1], reverse=True)
        
        for i, (node_id, degree) in enumerate(degrees[:20], 1):
            node_name = policy_graph.nodes[node_id].get('id', node_id)
            print(f"   {i:2d}. {node_name[:50]:<50} (degree: {degree})")
    else:
        print("   Nenhum critério encontrado")

# 6. Exemplos de trilhas (paths)
print("\n6️⃣ EXEMPLOS DE TRILHAS (10 amostras aleatórias da raiz até folha)")

def gerar_trilhas_aleatorias(node: Dict[str, Any], path: List[str] = None, max_trilhas: int = 10) -> List[List[str]]:
    """Gera trilhas aleatórias da raiz até folhas."""
    if path is None:
        path = []
    
    path = path + [node.get('pergunta', node.get('classe', node.get('id', 'N/A'))[:50])]
    
    if node.get('tipo') == 'terminal':
        return [path]
    
    all_paths = []
    for subnode in node.get('subnodos', []):
        all_paths.extend(gerar_trilhas_aleatorias(subnode, path, max_trilhas))
        if len(all_paths) >= max_trilhas:
            break
    
    return all_paths[:max_trilhas]

trilhas = gerar_trilhas_aleatorias(arvore_decisao, max_trilhas=10)

for i, trilha in enumerate(trilhas, 1):
    print(f"\n   Trilha {i}:")
    for j, step in enumerate(trilha):
        indent = "   " * (j + 1)
        print(f"{indent}{'└─' if j == len(trilha) - 1 else '├─'} {step}")

print("\n" + "="*80)

## [10] Smoke Test Local

Valida a árvore JSON com um evento fictício de exemplo.

In [ ]:
print("\n" + "="*80)
print("🧪 SMOKE TEST - Validação da Árvore")
print("="*80)

# Evento de exemplo
evento_exemplo = """
Vazamento de 15m³ de óleo diesel durante operação de abastecimento de embarcação.
Houve contaminação de solo e pequeno impacto em corpo hídrico próximo.
Nenhum trabalhador ferido. Operação de contenção realizada em 4 horas.
"""

print(f"\n📝 Evento de Exemplo:")
print(evento_exemplo)

print("\n🔍 Navegação Manual pela Árvore:\n")

# Função auxiliar para navegação
def navegar_arvore_manual(node: Dict[str, Any], level: int = 0):
    """Exibe estrutura da árvore para navegação manual."""
    indent = "  " * level
    
    if node.get('tipo') == 'terminal':
        print(f"{indent}🏁 TERMINAL: {node.get('classe', 'N/A')}")
        return
    
    print(f"{indent}❓ {node.get('pergunta', 'N/A')}")
    
    # Mostrar primeiras 3 opções
    for i, subnode in enumerate(node.get('subnodos', [])[:3], 1):
        print(f"{indent}   {i}. Opção: {subnode.get('pergunta', subnode.get('classe', subnode.get('id', 'N/A')))}")
    
    if len(node.get('subnodos', [])) > 3:
        print(f"{indent}   ... (mais {len(node['subnodos']) - 3} opções)")

# Exibir estrutura da raiz
print("RAIZ:")
navegar_arvore_manual(arvore_decisao)

# Simular seleção de caminho baseado no evento
print("\n🎯 Caminho Simulado (baseado no evento):")
print("\n1. Pergunta: 'Qual o tipo de ocorrência?'")
print("   Resposta: 'Acidente com Impacto no Meio Ambiente' (baseado em 'vazamento', 'contaminação')")

# Se houver subnodos, navegar pelo primeiro
if arvore_decisao.get('subnodos'):
    # Tentar encontrar nó relacionado a meio ambiente
    meio_ambiente_node = None
    for subnode in arvore_decisao['subnodos']:
        pergunta = subnode.get('pergunta', '').lower()
        if 'ambiente' in pergunta or 'meio' in pergunta:
            meio_ambiente_node = subnode
            break
    
    if not meio_ambiente_node:
        meio_ambiente_node = arvore_decisao['subnodos'][0]
    
    print(f"\n2. Nó selecionado: {meio_ambiente_node.get('pergunta', meio_ambiente_node.get('id'))}")
    
    if meio_ambiente_node.get('subnodos'):
        print(f"   Próximas perguntas disponíveis:")
        for i, sub in enumerate(meio_ambiente_node['subnodos'][:3], 1):
            print(f"      {i}. {sub.get('pergunta', sub.get('classe', 'N/A'))}")

print("\n✅ Validação da estrutura JSON:")
print("   - Raiz possui campo 'id': ✅" if 'id' in arvore_decisao else "   - Raiz FALTA campo 'id': ❌")
print("   - Raiz possui campo 'pergunta': ✅" if 'pergunta' in arvore_decisao else "   - Raiz FALTA 'pergunta': ❌")
print("   - Raiz possui campo 'tipo': ✅" if 'tipo' in arvore_decisao else "   - Raiz FALTA 'tipo': ❌")
print("   - Raiz possui 'subnodos': ✅" if 'subnodos' in arvore_decisao else "   - Raiz FALTA 'subnodos': ❌")

# Validar estrutura recursivamente
def validar_estrutura(node: Dict[str, Any], path: str = "raiz") -> List[str]:
    """Valida estrutura da árvore recursivamente."""
    erros = []
    
    if 'id' not in node:
        erros.append(f"{path}: Falta campo 'id'")
    
    if 'tipo' not in node:
        erros.append(f"{path}: Falta campo 'tipo'")
    elif node['tipo'] == 'terminal':
        if 'classe' not in node:
            erros.append(f"{path}: Nó terminal sem campo 'classe'")
    elif node['tipo'] == 'decisao':
        if 'pergunta' not in node:
            erros.append(f"{path}: Nó de decisão sem campo 'pergunta'")
        if 'subnodos' not in node:
            erros.append(f"{path}: Nó de decisão sem campo 'subnodos'")
        else:
            for i, subnode in enumerate(node['subnodos']):
                erros.extend(validar_estrutura(subnode, f"{path}/subnodo[{i}]"))
    
    return erros

erros_estrutura = validar_estrutura(arvore_decisao)

if erros_estrutura:
    print("\n⚠️ Erros de estrutura encontrados:")
    for erro in erros_estrutura[:10]:  # Mostrar primeiros 10
        print(f"   - {erro}")
    if len(erros_estrutura) > 10:
        print(f"   ... (mais {len(erros_estrutura) - 10} erros)")
else:
    print("\n✅ Estrutura da árvore válida!")

print("\n" + "="*80)

## 📦 Resumo dos Artefatos Gerados

Todos os artefatos foram salvos em `artifacts/`:

1. **`anp_text_corpus.jsonl`** - Corpus de textos limpos (um documento por linha)
2. **`anp_kg.graphml`** - Knowledge Graph completo (formato GraphML)
3. **`anp_kg.json`** - Knowledge Graph completo (formato JSON)
4. **`anp_policy.graphml`** - Policy Graph decisório (formato GraphML)
5. **`anp_policy.json`** - Policy Graph decisório (formato JSON)
6. **`anp_tree.json`** - Árvore de decisão final (compatível com classificador)

### Próximos Passos

1. **Validação manual**: Revisar `anp_tree.json` para garantir coerência normativa
2. **Refinamento do KG**: Ajustar prompt guia e reprocessar chunks com melhor qualidade
3. **Integração com LATS**: Carregar `anp_tree.json` no classificador LATS-P
4. **Expansão de classes**: Adicionar mais informações normativas aos nós terminais
5. **Teste com eventos reais**: Validar árvore com casos de uso da ANP

### Notas Importantes

- ⚠️ **MODO DE TESTE ATIVO**: Apenas 10 chunks foram processados. Para produção, altere `TEST_MODE = False`
- 💰 **Custo de API**: Processar corpus completo pode custar significativamente em tokens Azure OpenAI
- 🔄 **Reexecução**: Notebook é idempotente - pode ser reexecutado para regenerar artefatos
- 📊 **Qualidade**: Qualidade final depende da qualidade dos PDFs e do prompt guia

In [ ]:
# Exibir localização dos artefatos
print("\n📦 ARTEFATOS GERADOS:\n")
print(f"   📄 Corpus:        {CORPUS_FILE}")
print(f"   🕸️  KG (GraphML):  {KG_GRAPHML}")
print(f"   🕸️  KG (JSON):     {KG_JSON}")
print(f"   🎯 Policy (GraphML): {POLICY_GRAPHML}")
print(f"   🎯 Policy (JSON):    {POLICY_JSON}")
print(f"   🌳 Árvore (JSON):    {TREE_JSON}")
print("\n✅ Pipeline completo!")